In [2]:

# STEP 0. 라이브러리 로딩
#   - 지금은 "기본 정보(판매순위, 상품명, 가격, 별점, 리뷰수)만"
#     크롤링해서 엑셀 뼈대 만드는 단계

import urllib.request , urllib              # (옵션) 요청 관련 모듈 (거의 안 써도 됨)
import pandas as pd                         # 표(데이터프레임) / 엑셀 저장
from bs4 import BeautifulSoup               # HTML 파싱
import os, tempfile, time, math, random     # 운영체제 / 임시폴더 / 시간 / 수학 / 랜덤
import undetected_chromedriver as uc        # ★★★ 크롬 자동조작 (봇탐지 회피 버전)
from selenium.webdriver.chrome.options import Options   # 크롬 옵션
from selenium.webdriver.common.by import By             # 요소 찾기 기준 (By.CSS_SELECTOR 등)
from selenium.webdriver.support.ui import WebDriverWait # 명시적 대기
from selenium.webdriver.support import expected_conditions as EC  # 대기 조건
from selenium.common.exceptions import WebDriverException         # 예외 처리용
from openpyxl import load_workbook             # 엑셀 엔진 (to_excel에서 사용)
import requests                                # (옵션) HTTP 요청
import re                                      # 정규표현식 (숫자만 뽑기 등)

# STEP 1. 사용자 입력 (크롤링 개수 / 저장 폴더)
print("=" * 80)
print(" 올리브영 Best상품 기본정보(순위+상품명+가격+별점+리뷰수) 뼈대 만들기 ")
print("=" * 80)

# 크롤링할 상품 개수 (예: 100 입력)
cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))   # ← int()로 숫자로 변환
page_cnt = math.ceil(cnt / 60)                        # (참고) 60개당 1페이지라고 가정

# 저장 폴더 입력 (기본값: c:\py_temp\ )
f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본경로:c:\\py_temp\\):")
if f_dir == '':                                      # 아무 것도 안 쳤을 때
    f_dir = "c:\\py_temp\\"

print("\n")
if cnt > 60:
    print("요청 건수가 많아서 시간이 제법 소요됩니다. 잠시만 기다려 주세요~~")
else:
    print("요청하신 데이터를 수집하고 있습니다. 잠시만 기다려 주세요~~")


# STEP 2. 저장할 폴더/파일 이름 만들기 (타임스탬프 사용)
sec_name = '코스메틱'            # 섹션 이름 (폴더/파일 이름에 포함)
query_txt = '올리브영'           # 검색 키워드 느낌 (폴더/파일 이름에 포함)

# 현재 시간으로 폴더/파일 구분
n = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (
    n.tm_year, n.tm_mon, n.tm_mday,
    n.tm_hour, n.tm_min, n.tm_sec
)

# 최종 저장 폴더: c:\py_temp\타임스탬프-올리브영-코스메틱\
save_folder = f_dir + s1 + '-' + query_txt + '-' + sec_name
os.makedirs(save_folder, exist_ok=True)      # 폴더 없으면 생성
os.chdir(save_folder)                        # 현재 작업 폴더 변경

# 각 파일 경로 (txt / csv / xlsx) 만들기
ff_dir  = save_folder
ff_name = save_folder + '\\' + s1 + '-' + query_txt + '-' + sec_name + '.txt'
fc_name = save_folder + '\\' + s1 + '-' + query_txt + '-' + sec_name + '.csv'
fx_name = save_folder + '\\' + s1 + '-' + query_txt + '-' + sec_name + '.xlsx'

# 전체 수행 시간 측정을 위한 시작 시간
s_time = time.time()


# STEP 3. 사람처럼 기다리고 스크롤하는 함수 (봇탐지 회피용)★
def human_sleep(min_sec=1, max_sec=3):
    """랜덤 대기로 봇 감지 회피"""
    time.sleep(random.uniform(min_sec, max_sec))

def human_scroll(driver, scroll_pause_min=1.5, scroll_pause_max=3.5):
    """사람처럼 불규칙하게 스크롤"""
    scroll_amount = random.randint(800, 1500)                     # 800~1500px 랜덤 스크롤
    driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
    human_sleep(scroll_pause_min, scroll_pause_max)               # 스크롤 후 잠깐 대기


# 드라이브 열리기전에 함수정의하기
def get_price_rating_review(driver, detail_url):
    """상세페이지 1개에 들어가서 가격 / 별점 / 리뷰수를 가져오는 함수"""

    # 1) 상세페이지 접속
    driver.get(detail_url)          # ★ 해당 상품 상세 페이지로 이동
    human_sleep(2, 3)               # ★ 로딩 잠깐 대기

    # 2) 페이지 로딩 대기 (필요하면 조건 추가 하기)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.ID, "main")    # ★ 메인 영역 뜰 때까지 기다리기
            )
        )
    except:
        pass   # 너무 빡세게 안 잡고 넘어가기

    # ------------------------------
    # (1) 가격
    try:
        price_elem = driver.find_element(
            By.XPATH,
            '//*[@id="main"]/div[2]/div/div[2]/div/div[1]/div[3]/div/div/span/span[1]'  
        )
        raw_price = price_elem.text.strip()         # 예: "11,500"
        price = re.sub(r"[^0-9]", "", raw_price)    # 숫자만 남기기 → "11500"
    except:
        price = ""                                  # 못 찾으면 빈칸

    # ------------------------------
    # (2) 별점
    try:
        rating_elem = driver.find_element(
            By.XPATH,
            '//*[@id="main"]/div[2]/div/div[2]/div/div[1]/div[5]/div/div[1]/span'   
        )
        raw_rating = rating_elem.text.strip()       # 예: "평점4.8" 혹은 "4.8"
        nums = re.findall(r"\d+\.\d+|\d+", raw_rating)  # "4.8"만 뽑기
        rating = nums[0] if nums else ""
    except:
        rating = ""

    # ------------------------------
    # (3) 리뷰수
    try:
        review_elem = driver.find_element(
            By.XPATH,
            '//*[@id="main"]/div[2]/div/div[2]/div/div[1]/div[5]/div/div[2]/button/span'   
        )
        raw_review = review_elem.text.strip()       # 예: "5,027"
        review_cnt = re.sub(r"[^0-9]", "", raw_review)  # "5027"
    except:
        review_cnt = ""

    # 4) 결과 반환 (튜플)
    return price, rating, review_cnt


# STEP 4. 크롬 드라이버 설정 및 브라우저 오픈
print("\n🔧 Chrome 드라이버 설정 중...")

CHROME_PATH   = r"C:\Program Files\Google\Chrome\Application\chrome.exe"  # 네 PC 크롬 경로
VERSION_MAIN  = 142                                                       # 크롬 메이저 버전

options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')        # 창 최대화

# undetected_chromedriver용 임시 프로필 폴더 (봇탐지 더 회피)
user_data = os.path.join(tempfile.gettempdir(), f"uc_profile_{int(time.time())}")
os.makedirs(user_data, exist_ok=True)

print("🚀 Chrome 브라우저 시작 중...")

try:
    # ★★★ undetected_chromedriver 로 크롬 실행
    driver = uc.Chrome(
        options=options,
        browser_executable_path=CHROME_PATH,
        version_main=VERSION_MAIN,
        user_data_dir=user_data,
        use_subprocess=False,
        headless=False,         # 실제 브라우저 보이게
    )
    print("✅ Chrome 드라이버 초기화 완료!")
except Exception as e:
    print(f"❌ 드라이버 초기화 실패: {str(e)}")
    # 최소 설정으로 재시도
    driver = uc.Chrome(
        browser_executable_path=CHROME_PATH,
        version_main=VERSION_MAIN,
    )


# (선택) 네이버로 한번 접속해서 브라우저 워밍업
print("\n📡 브라우저 워밍업 중...")
try:
    driver.get("https://www.naver.com")
    human_sleep(2, 3)
    print("✅ 워밍업 완료!")
except:
    pass

# STEP 5. 올리브영 베스트 100 페이지로 이동
print("\n📡 올리브영 메인 페이지 접속 중...")
try:
    driver.get("https://www.oliveyoung.co.kr")
    human_sleep(3, 5)
    print("✅ 메인 페이지 접속 완료!")

    # 사람처럼 살짝 스크롤
    for i in range(2):
        driver.execute_script(f"window.scrollBy(0, {random.randint(300, 600)});")
        human_sleep(1, 2)
except:
    pass

print("\n📡 베스트 100 페이지로 이동 중...")

#올리브영 랭킹사이트
target_url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EC%A0%84%EC%B2%B4"

try:
    driver.get(target_url)
    human_sleep(4, 6)
    if "oliveyoung" in driver.current_url:
        print("✅ 올리브영 베스트 페이지 접속 성공!")
    else:
        print("페이지가 제대로 안 떴으면 수동으로 이동 후 Enter...")
        input()
except:
    print("페이지가 제대로 안 떴으면 수동으로 이동 후 Enter...")
    input()

print("\n⏳ 페이지 로딩 대기 중...")
human_sleep(3, 5)

# 상품 리스트 ul이 뜰 때까지 최대 20초 대기
try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ul.cate_prd_list"))
    )
    print("✅ 상품 리스트 로드 완료!")
except:
    print("상품이 보이면 Enter...")
    input()


# STEP 6. 아래로 스크롤해서 상품들을 충분히 로딩
print("\n📜 스크롤하여 상품 로딩 중...")

last_height    = driver.execute_script("return document.body.scrollHeight")  # 현재 페이지 높이
scroll_count   = 0
no_change_count = 0

while scroll_count < 13:                      # 13번스크롤하기
    human_scroll(driver, 1.5, 3.5)           # 사람처럼 랜덤 스크롤+대기

    # 가끔 위로 살짝 스크롤 (사람스러운 행동)
    if random.random() < 0.15:
        scroll_back = random.randint(-300, -100)
        driver.execute_script(f"window.scrollBy(0, {scroll_back});")
        human_sleep(0.5, 1.5)

    new_height = driver.execute_script("return document.body.scrollHeight")
    scroll_count += 1

    if scroll_count % 3 == 0:
        print(f"  스크롤 {scroll_count}회 완료...")

    # 더 이상 높이가 안 늘어나면 no_change_count 증가
    if new_height == last_height:
        no_change_count += 1
        if no_change_count >= 4:
            print("  📌 더 이상 로드할 상품이 없습니다.")
            break
    else:
        no_change_count = 0

    last_height = new_height

# 스크롤 다 했으니 페이지 상단으로 올리기
print("  페이지 상단으로 이동 중...")
for i in range(5):
    scroll_up = -last_height // 5
    driver.execute_script(f"window.scrollBy(0, {scroll_up});")
    human_sleep(0.3, 0.8)

human_sleep(2, 3)


# STEP 7. HTML 파싱해서 상품 리스트(li) 전부 찾기

print("\n🔍 HTML 파싱 중...")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 1차 시도: ul.cate_prd_list 아래 li
product_list = soup.select("ul.cate_prd_list li")

# 혹시 못 찾으면 보조 셀렉터
if not product_list:
    product_list = soup.select("div.prd_list li")

print(f"\n✅ 총 {len(product_list)}개 상품 발견!")

if len(product_list) < 10:
    print("상품이 너무 적게 잡혔습니다. 화면 확인 후 Enter...")
    input()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    product_list = soup.select("ul.cate_prd_list li")

print(f"📊 요청하신 {cnt}개 수집 시작...\n")
print("=" * 100)

# ============================================================
# STEP 8. for문 돌면서 "기본 정보"만 수집
#   - 판매순위 : 인덱스(랭킹순위중 오특 이있어서 인덱스번호로 쓰기로함)
#   - 상품명   : 리스트 페이지에서 [브랜드 + 상품명]
#   - 상세내용 : 지금 단계에서는 ''(빈 문자열)로 채우기
#   - 판매가격 : 상세페이지에서 XPATH로 가져오기
#   - 별점     : 상세페이지에서 XPATH로 가져오기
#   - 리뷰수   : 상세페이지에서 XPATH로 가져오기
# ============================================================
rank_list        = []   # 판매순위
name_list        = []   # 상품명
detail_list      = []   # 상세내용(지금은 빈 칸)
price_list       = []   # 판매가격
rating_list      = []   # 별점
review_cnt_list  = []   # 리뷰수

count = 0   # 실제 수집한 상품 개수

for idx, item in enumerate(product_list, start=1):
    # 요청한 개수(cnt)만큼만 수집
    if count >= cnt:
        break

    try:
        # ----------------------------------------
        # (1) 판매순위 : 인덱스 번호 사용 (1,2,3...)
        # ----------------------------------------
        rank = idx                    # 1,2,3,... 순위
        rank_list.append(rank)

        # ----------------------------------------
        # (2) 상품명 : 리스트에서 [브랜드 + 공백 + 상품명]
        #     span.tx_brand  +  p.tx_name
        # ----------------------------------------
        brand_elem = item.select_one("span.tx_brand")   # ★ 브랜드
        title_elem = item.select_one("p.tx_name")       # ★ 상품명

        brand = brand_elem.get_text(strip=True) if brand_elem else ""
        title = title_elem.get_text(strip=True) if title_elem else ""

        product_name = f"{brand} {title}".strip()       # "CJ이너비 슬리밍샷..."
        name_list.append(product_name)

        # ----------------------------------------
        # (3) 상세페이지 URL 찾기 (a 태그 href)
        # ----------------------------------------
        link_elem = item.select_one("a[data-ref-goodsno]")   # ★ 우선순위: data-ref-goodsno 있는 a
        if not link_elem:
            link_elem = item.select_one("a")                 # ★ 없으면 그냥 첫 번째 a

        if link_elem and link_elem.get("href"):
            href = link_elem.get("href").strip()
            if href.startswith("http"):
                detail_url = href
            else:
                detail_url = "https://www.oliveyoung.co.kr" + href
        else:
            detail_url = ""

        # ----------------------------------------
        # (4) 상세내용 : 1단계에서는 아직 비워두기 ("")
        #     → 나중에 OCR 결과를 여기에 채울 예정
        # ----------------------------------------
        detail_list.append("")

        # ----------------------------------------
        # (5) 가격 / 별점 / 리뷰수 : 상세페이지에서 가져오기
        # ----------------------------------------
        if detail_url:
            price, rating, review_cnt = get_price_rating_review(driver, detail_url)
        else:
            price, rating, review_cnt = "", "", ""

        price_list.append(price)
        rating_list.append(rating)
        review_cnt_list.append(review_cnt)

        # ----------------------------------------
        # (6) 진행상황 출력
        # ----------------------------------------
        count += 1
        print(
            f"[{count:3d}] 순위: {rank:3d} | "
            f"상품명: {product_name[:40]} | "
            f"가격: {price} | 별점: {rating} | 리뷰수: {review_cnt}"
        )

        if count % 5 == 0:
            print("\n" + "=" * 100)
            print(f"  ✅ {count}개 완료! ({count}/{cnt})")
            print("=" * 100 + "\n")
            human_sleep(1, 2)

    except Exception as e:
        print(f"     ❌ 오류 발생: {str(e)}")
        continue


# ============================================================
# STEP 9. 브라우저 종료
# ============================================================
print(f"\n{'=' * 100}")
print(f"✅ 총 {count}개 상품 기본 정보 수집 완료!")
print(f"{'=' * 100}\n")

driver.quit()
print("🔒 브라우저 종료\n")

# ============================================================
# STEP 10. DataFrame 생성 (엑셀 뼈대 만들기)
#   컬럼: [판매순위] [상품명] [상세내용] [판매가격] [별점] [리뷰수]
# ============================================================
df = pd.DataFrame({
    '판매순위': rank_list,        # 인덱스 기반 순위
    '상품명':   name_list,        # 상품 이름
    '상세내용': detail_list,      # 지금은 모두 "" (빈 칸)
    '판매가격': price_list,       # 가격 (숫자 문자열)
    '별점':     rating_list,      # 별점 (문자열 그대로)
    '리뷰수':   review_cnt_list   # 리뷰 개수 (숫자 문자열)
})

# ============================================================
# STEP 11. CSV / 엑셀 / TXT 저장
# ============================================================
# 1) CSV 저장
try:
    df.to_csv(fc_name, encoding='utf-8-sig', index=False)
    print(f"✅ CSV 저장 완료 → {fc_name}")
except Exception as e:
    print(f"❌ CSV 저장 실패: {str(e)}")

# 2) 엑셀 저장
try:
    df.to_excel(fx_name, index=False, engine='openpyxl')
    print(f"✅ Excel 저장 완료 → {fx_name}")
except Exception as e:
    print(f"❌ Excel 저장 실패: {str(e)}")

# 3) TXT 저장 (간단 요약)
try:
    with open(ff_name, 'w', encoding='utf-8') as f:
        f.write("=" * 100 + "\n")
        f.write("올리브영 베스트 기본정보(판매순위+상품명+가격+별점+리뷰수)\n")
        f.write("=" * 100 + "\n\n")
        for i in range(len(df)):
            f.write(f"[{df.iloc[i]['판매순위']:>3}] {df.iloc[i]['상품명']}\n")
            f.write(f"  가격: {df.iloc[i]['판매가격']} / 별점: {df.iloc[i]['별점']} / 리뷰수: {df.iloc[i]['리뷰수']}\n")
            f.write("-" * 100 + "\n\n")
    print(f"✅ TXT 저장 완료 → {ff_name}")
except Exception as e:
    print(f"❌ TXT 저장 실패: {str(e)}")

# ============================================================
# STEP 12. 소요 시간 / 요약 출력
# ============================================================
e_time = time.time()
t_time = e_time - s_time

print(f"\n{'=' * 100}")
print("🎉 1단계: 올리브영 기본 정보 크롤링(뼈대) 완료!")
print(f"⏱️  소요 시간: {t_time:.2f}초 ({t_time/60:.1f}분)")
print(f"📊 수집 건수: {count}개")
print(f"📁 저장 폴더: {ff_dir}")
print(f"\n📋 저장 파일:")
print(f"  - Excel: {fx_name}")
print(f"  - CSV  : {fc_name}")
print(f"  - TXT  : {ff_name}")
print(f"{'=' * 100}")
print("\n📊 DataFrame 미리보기:")
print(df.head(10))


 올리브영 Best상품 기본정보(순위+상품명+가격+별점+리뷰수) 뼈대 만들기 


1.크롤링 할 건수는 몇건입니까?:  100
2.파일을 저장할 폴더명만 쓰세요(기본경로:c:\py_temp\): 




요청 건수가 많아서 시간이 제법 소요됩니다. 잠시만 기다려 주세요~~

🔧 Chrome 드라이버 설정 중...
🚀 Chrome 브라우저 시작 중...
✅ Chrome 드라이버 초기화 완료!

📡 브라우저 워밍업 중...
✅ 워밍업 완료!

📡 올리브영 메인 페이지 접속 중...
✅ 메인 페이지 접속 완료!

📡 베스트 100 페이지로 이동 중...
✅ 올리브영 베스트 페이지 접속 성공!

⏳ 페이지 로딩 대기 중...
✅ 상품 리스트 로드 완료!

📜 스크롤하여 상품 로딩 중...
  스크롤 3회 완료...
  📌 더 이상 로드할 상품이 없습니다.
  페이지 상단으로 이동 중...

🔍 HTML 파싱 중...

✅ 총 100개 상품 발견!
📊 요청하신 100개 수집 시작...

[  1] 순위:   1 | 상품명: 마미케어 [펀딩특가/3천원페이백]마미케어 EMS브이쎄라(+펀딩기프트증정) | 가격: 79000 | 별점: 4.6 | 리뷰수: 817
[  2] 순위:   2 | 상품명: CJ이너비 [2025 어워즈] CJ 이너비 슬리밍샷 5+1병 어워즈 기획 | 가격: 11500 | 별점: 4.8 | 리뷰수: 5027
[  3] 순위:   3 | 상품명: 에스네이처 [12/10 하루특가/3년연속수상/수분천재크림] 에스네이처 아 | 가격: 21000 | 별점: 4.8 | 리뷰수: 56714
[  4] 순위:   4 | 상품명: 파넬 [25년 마지막 특가/한정기획 소진임박] 파넬 시카마누 92세럼 3 | 가격: 22900 | 별점: 4.8 | 리뷰수: 7943
[  5] 순위:   5 | 상품명: 토리든 [12/10하루특가][2025 어워즈 1위] 토리든 다이브인 저분 | 가격: 31900 | 별점: 4.8 | 리뷰수: 77441

  ✅ 5개 완료! (5/100)

[  6] 순위:   6 | 상품명: 메디힐 [2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크 | 가격: 10000 | 별점: 4.9 | 리뷰수: 272322
[  7] 순위:   7 